In [1]:
import panel as pn
pn.extension('tabulator', template='material', sizing_mode='stretch_width')

import pyvo as vo
import datetime as dt
from astropy.time import Time
import param
import numpy as np

In [2]:
url = "https://vo.astro.rug.nl/tap"
service = vo.dal.TAPService(url)

In [3]:
column_names = np.array([])
for i in service.tables['observations.raw'].columns:
    column_names = np.append(column_names, i.name)
print(column_names)
['kw_date_obs', 'kw_imagetyp', 'id']

['id' 'obs_jd' 'ra' 'dec' 'filename' 'file_id' 'd_plate_scale' 'd_odds'
 'kw_imagetyp' 'kw_date_obs' 'kw_object' 'kw_objctra' 'kw_objctdec'
 'kw_filter' 'kw_expstate' 'kw_exposure' 'kw_airmass' 'kw_simple'
 'kw_bitpix' 'kw_naxis' 'kw_naxis1' 'kw_naxis2' 'kw_bscale' 'kw_bzero'
 'kw_bias' 'kw_focallen' 'kw_aptarea' 'kw_aptdia' 'kw_time_obs'
 'kw_swcreate' 'kw_set_temp' 'kw_colorccd' 'kw_dispcolr' 'kw_ccdsfpt'
 'kw_xorgsubf' 'kw_yorgsubf' 'kw_ccdsubfl' 'kw_ccdsubft' 'kw_xbinning'
 'kw_ccdxbin' 'kw_ybinning' 'kw_ccdybin' 'kw_ccd_temp' 'kw_temperat'
 'kw_teltkra' 'kw_teltkdec' 'kw_centaz' 'kw_centalt' 'kw_telha' 'kw_lst'
 'kw_sitelat' 'kw_sitelong' 'kw_instrume' 'kw_egain' 'kw_e_gain'
 'kw_xpixsz' 'kw_ypixsz' 'kw_sbigimg' 'kw_user_2' 'kw_datamax'
 'kw_sbstdver' 'kw_exptime' 'kw_cblack' 'kw_cwhite' 'kw_ctype1'
 'kw_ctype2' 'kw_equinox' 'kw_crval1' 'kw_crval2' 'kw_crpix1' 'kw_crpix2'
 'kw_cunit1' 'kw_cunit2' 'kw_cd1_1' 'kw_cd1_2' 'kw_cd2_1' 'kw_cd2_2'
 'kw_ap_0_0' 'kw_ap_0_1' 'kw_ap_0_2' 'kw_

['kw_date_obs', 'kw_imagetyp', 'id']

In [22]:
start_date = dt.datetime(2020, 5, 1, 12, 0, 0)
end_date = dt.datetime(2020, 6, 1, 12, 0, 0)

class CoolTable(param.Parameterized):
    rows = param.Integer(default=10, bounds=(0, 19))
    dates = param.DateRange((start_date, end_date))

    @param.depends('dates', watch=True)
    def data(self):
        query = """
        SELECT TOP 100 filename, ra, dec, kw_date_obs
        FROM observations.raw
        WHERE kw_imagetyp = 'Light Frame'
        AND ra IS NOT NULL
        AND obs_jd BETWEEN {} AND {}
        ORDER BY obs_jd
        """.format(Time(self.dates[0]).jd, Time(self.dates[1]).jd)
        result = service.search(query)
        data_table = result.to_table().to_pandas()
        return data_table

    @param.output('table_data')
    def output(self):
        return self.table

    @param.depends('data', 'rows')
    def table(self):
        return self.data().iloc[:self.rows].style.hide()

    def panel(self):
        settings = pn.Row(
            pn.Param(self, widgets={
                'rows': pn.widgets.IntSlider,
                'dates': pn.widgets.DatetimeRangePicker
            }, width=300, sizing_mode="fixed", name="Settings")
        )
        bootstrap.sidebar.append(settings)
        bootstrap.main.append(
            pn.Column(
                '## Title',
                self.table
            )
        )
        return bootstrap

In [101]:
options = {'ID': 'id', 'Observation date': 'kw_date_obs', 'Image type': 'kw_imagetyp', 'Right ascension': 'ra', 'Declination': 'dec'}
class CompileQuery(param.Parameterized):
    rows = param.Integer(default=10, bounds=(0, 19))
    dates = param.DateRange((dt.datetime(2020, 4, 1), dt.datetime(2020, 4, 30)))
    select = param.ListSelector(default=['ID', 'Observation date', 'Right ascension', 'Declination'], objects=list(options.keys()))
    ra_not_null = param.Boolean(default=True)
    dec_not_null = param.Boolean(default=True)

    start_query = """SELECT TOP 100 """

    @param.depends('select', watch=True)
    def select_query(self):
        query = """"""
        for i in self.select:
            query += options[i] + ','
        return query.rstrip(',')

    def from_query(self):
        query = """ FROM observations.raw"""
        return query

    @param.depends('ra_not_null', watch=True)
    def ra_check(self):
        if self.ra_not_null:
            query = """ WHERE ra IS NOT NULL """
            return query
        else:
            query = """ """
            return query

    @param.depends('dec_not_null', 'ra_not_null', watch=True)
    def dec_check(self):
        if self.ra_not_null:
            if self.dec_not_null:
                query = """AND dec IS NOT NULL """
                return query
            else:
                query = """"""
                return query
        else:
            if self.dec_not_null:
                query = """WHERE dec IS NOT NULL """
                return query
            else:
                query = """"""
                return query

    @param.depends('dates', watch=True)
    def date_query(self):
        if self.dec_check().find('WHERE') != -1 or self.ra_check().find('WHERE') != -1:
            query = """AND """
        else:
            query = """WHERE """
        query += """obs_jd BETWEEN {} AND {} ORDER BY obs_jd""".format(Time(self.dates[0]).jd, Time(self.dates[1]).jd)
        return query

    def data(self):
        query = self.start_query + self.select_query() + self.from_query() + self.ra_check() + self.dec_check() + self.date_query()
        result = service.search(query)
        data_table = result.to_table().to_pandas()
        return data_table

    # def data(self):
    #     query = self.start_query + self.select_query() + self.from_query() + self.ra_check() + self.dec_check() + self.date_query()
    #     # result = service.search(query)
    #     # data_table = result.to_table().to_pandas()
    #     return query

In [102]:
example = CompileQuery()
print(example.data())

       id              kw_DATE_OBS          ra        dec
0   42252  2020-04-15T17:58:30.844  118.560275   0.535831
1   42189  2020-04-15T17:58:58.975  118.556108   0.535822
2   42287  2020-04-15T17:59:28.231  118.556108   0.535822
3   42265  2020-04-15T17:59:57.250  118.556108   0.535822
4   42225  2020-04-15T18:00:26.356  118.560275   0.535831
..    ...                      ...         ...        ...
95  42183  2020-04-16T00:36:13.844  328.425108  62.606736
96   4600  2020-04-16T00:36:53.669  328.252961  62.658754
97  42206  2020-04-16T00:36:53.669  328.425108  62.606736
98  42187  2020-04-16T00:37:34.792  328.425108  62.606736
99   4606  2020-04-16T00:37:34.792  328.418657  62.600086

[100 rows x 4 columns]


In [103]:
class CoolerTable(CompileQuery):
    @param.depends('data', 'rows')
    def table(self):
        return self.data().iloc[:self.rows].style.hide()

    def panel(self):
        settings = pn.Row(
            pn.Param(self, widgets={
                'rows': pn.widgets.IntSlider,
                'dates': pn.widgets.DatetimeRangePicker,
                'select': {'widget_type': pn.widgets.CrossSelector, 'definition_order': False},
                'ra_not_null': pn.widgets.Checkbox,
                'dec_not_null': pn.widgets.Checkbox
            }, width=300, sizing_mode="fixed", name="Settings")
        )
        bootstrap.sidebar.append(settings)
        bootstrap.main.append(
            pn.Column(
                '## Title',
                self.table
            )
        )
        return bootstrap

In [104]:
bootstrap = pn.template.BootstrapTemplate(title='Bootstrap Template')

table_example = CoolerTable()
table_example.panel().show()

Launching server at http://localhost:52219


In [27]:
start = """SELECT TOP 10 """
select = 'kw_date_obs'
date = """
        FROM observations.raw
        WHERE ra IS NOT NULL
        AND obs_jd BETWEEN {} AND {}
        ORDER BY obs_jd
        """.format(Time(dt.datetime(2020, 5, 1, 12, 0, 0)).jd, Time(dt.datetime(2020, 6, 1, 12, 0, 0)).jd)

query = start + select + date
result = service.search(query)
data_table = result.to_table().to_pandas()
print(data_table)

               kw_DATE_OBS
0  2020-05-05T18:53:13.276
1  2020-05-05T18:53:17.165
2  2020-05-05T18:53:21.183
3  2020-05-05T18:53:25.104
4  2020-05-05T18:53:29.023
5  2020-05-05T18:53:32.980
6  2020-05-05T18:53:36.951
7  2020-05-05T18:53:40.971
8  2020-05-05T18:53:44.949
9  2020-05-05T18:53:48.755
